2 by 2 XOR problem

In [ ]:
import torch
from omegaconf import OmegaConf

In [ ]:
# prepare XOR data
X = torch.tensor([[-1, -1], [-1, 1], [1, -1], [1, 1]]).float()
y = torch.tensor([[0], [1], [1], [0]])

cfg = """
net:
  _target_: src.models.components.eqprop_backbone.AnalogEP2
  _partial_: true
  batch_size: 1
  dims: [2,2,1]
  beta: 0.01
  solver:
    _target_: src.eqprop.solver.AnalogEqPropSolver
    _partial_: true
    amp_factor: 1.0
    beta: ${net.beta}
    strategy:
      _target_: src.eqprop.strategy.NewtonStrategy
      clip_threshold: 0.1
      amp_factor: ${net.solver.amp_factor}
      max_iter: 5
      atol: 1e-5
      activation:
        _target_: src.eqprop.eqprop_util.P3OTS
        Is: 1e-6
        Vth: 1
        Vl: 0
        Vr: 0"""

cfg = OmegaConf.create(cfg)

In [ ]:
from hydra.utils import instantiate

net = instantiate(cfg.net)
net = net(hyper_params={"bias": True})

In [ ]:
# train
import torch.nn.functional as F

from src.eqprop.eqprop_util import init_params

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss(reduction="sum")
net.apply(init_params(min_w=1e-6, max_w_gain=0.3))

for i in range(1000):
    x = X[i % 4].unsqueeze(0)
    optimizer.zero_grad()
    logit = net(x)
    yhat = F.sigmoid(logit)
    y_onehot = F.one_hot(y[i % 4], 2).float()
    loss = loss_fn(yhat, y_onehot)
    loss.backward()
    net.eqprop(x)
    optimizer.step()
    if i % 100 == 0:
        print(net.loss.item())

In [ ]:
for buf in net.named_buffers():
    print(buf[0], buf[1].shape)